In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Data/winequality-red-cleaned.csv', sep=',')

In [3]:
df.head()

,volatile acidity,residual sugar,chlorides,total sulfur dioxide,pH,sulphates,alcohol,quality,category
0,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246,5,Bad
1,1.967442,0.043416,0.223875,0.624363,-0.719933,0.128950,-0.584777,5,Bad
2,1.297065,-0.169427,0.096353,0.229047,-0.331177,-0.048089,-0.584777,5,Bad
3,-1.384443,-0.453218,-0.264960,0.411500,-0.979104,-0.461180,-0.584777,6,Good
4,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246,5,Bad


In [4]:
y = pd.DataFrame([0. if item == 'Good' else 1. for item in df['category']])
y = [0. if item == 'Good' else 1. for item in df['category']]
y[:5]

[1.0, 1.0, 1.0, 0.0, 1.0]

In [5]:
features_df = df.drop(['quality', 'category'], axis=1)

In [6]:
features_df.head()

,volatile acidity,residual sugar,chlorides,total sulfur dioxide,pH,sulphates,alcohol
0,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246
1,1.967442,0.043416,0.223875,0.624363,-0.719933,0.128950,-0.584777
2,1.297065,-0.169427,0.096353,0.229047,-0.331177,-0.048089,-0.584777
3,-1.384443,-0.453218,-0.264960,0.411500,-0.979104,-0.461180,-0.584777
4,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246


In [7]:
import torch

In [8]:
# TODO: pip install sklearn
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_df, y, test_size=0.2, random_state=42)
# X_train = features_df
# y_train = np.array(y)

/Users/kendall/Dropbox/Current_Projects/python/pytorch_tutorial/.direnv/python-3.5.2/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.activation import Softmax


In [24]:
learning_rate = 0.05

In [25]:
class SingleLayerNet(nn.Module):
    def __init__(self):
        super(SingleLayerNet, self).__init__()
        self.fc1 = nn.Linear(7, 2)

    def forward(self, x):
        x = F.softmax(self.fc1(x))
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net0 = SingleLayerNet()
optimizer0 = optim.SGD(net0.parameters(), lr=learning_rate, momentum=0.9)

In [26]:
class DoubleLayerNet(nn.Module):
    def __init__(self):
        super(DoubleLayerNet, self).__init__()
        self.fc1 = nn.Linear(7, 7)
        self.fc2 = nn.Linear(7, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = F.softmax(self.fc2(x))
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net1 = DoubleLayerNet()
optimizer1 = optim.SGD(net1.parameters(), lr=learning_rate, momentum=0.9)

In [27]:
batch_size = 200
epochs = 100
epochs_to_print = 10

In [28]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss

In [29]:
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix, accuracy_score

In [30]:
def train_with_net(net, optimizer, criterion, batch_size, 
                   epochs, epochs_to_print):
    for epoch in range(epochs): # loop over the dataset multiple times
        
        running_loss = 0.0
        for start, end in zip(range(0, len(X_train), batch_size), 
                              range(batch_size, len(X_train), batch_size)):
            # get the inputs
            
            inputs = torch.from_numpy(X_train[start:end].as_matrix())
            inputs = inputs.float()
            labels = torch.Tensor(y_train[start:end])
            labels = labels.long()
    
            # wrap them in Variable
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()        
            optimizer.step()
            
            # print statistics
            running_loss += loss.data[0]

            if epoch % epochs_to_print == epochs_to_print - 1:
                print(loss.data[0])
                test_outputs = net(Variable(torch.from_numpy(X_test.as_matrix()).float()))
                _, predicted = torch.max(test_outputs.data, 1)
                accuracy = accuracy_score(y_test, predicted.numpy())
                print('%d loss: %.3f, accuracy: %.3f' % (epoch+1, running_loss / epochs_to_print,
                      accuracy))
                running_loss = 0.0

    print('Finished Training')

In [32]:
net0 = SingleLayerNet()
net1 = DoubleLayerNet()

train_with_net(net0, optimizer0, criterion, batch_size, epochs, epochs_to_print)

0.6865259408950806
10 loss: 0.069, accuracy: 0.562
0.6905931234359741
10 loss: 0.069, accuracy: 0.562
0.6835200786590576
10 loss: 0.068, accuracy: 0.562
0.6705790162086487
10 loss: 0.067, accuracy: 0.562
0.6823490858078003
10 loss: 0.068, accuracy: 0.562
0.6706100702285767
10 loss: 0.067, accuracy: 0.562
0.6865259408950806
20 loss: 0.069, accuracy: 0.562
0.6905931234359741
20 loss: 0.069, accuracy: 0.562
0.6835200786590576
20 loss: 0.068, accuracy: 0.562
0.6705790162086487
20 loss: 0.067, accuracy: 0.562
0.6823490858078003
20 loss: 0.068, accuracy: 0.562
0.6706100702285767
20 loss: 0.067, accuracy: 0.562
0.6865259408950806
30 loss: 0.069, accuracy: 0.562
0.6905931234359741
30 loss: 0.069, accuracy: 0.562
0.6835200786590576
30 loss: 0.068, accuracy: 0.562
0.6705790162086487
30 loss: 0.067, accuracy: 0.562
0.6823490858078003
30 loss: 0.068, accuracy: 0.562
0.6706100702285767
30 loss: 0.067, accuracy: 0.562
0.6865259408950806
40 loss: 0.069, accuracy: 0.562
0.6905931234359741
40 loss: 0.0

In [33]:
train_with_net(net1, optimizer1, criterion, batch_size, epochs, epochs_to_print)

0.6897417306900024
10 loss: 0.069, accuracy: 0.547
0.6874913573265076
10 loss: 0.069, accuracy: 0.547
0.6825845837593079
10 loss: 0.068, accuracy: 0.547
0.6925886273384094
10 loss: 0.069, accuracy: 0.547
0.696567177772522
10 loss: 0.070, accuracy: 0.547
0.6879830956459045
10 loss: 0.069, accuracy: 0.547
0.6897417306900024
20 loss: 0.069, accuracy: 0.547
0.6874913573265076
20 loss: 0.069, accuracy: 0.547
0.6825845837593079
20 loss: 0.068, accuracy: 0.547
0.6925886273384094
20 loss: 0.069, accuracy: 0.547
0.696567177772522
20 loss: 0.070, accuracy: 0.547
0.6879830956459045
20 loss: 0.069, accuracy: 0.547
0.6897417306900024
30 loss: 0.069, accuracy: 0.547
0.6874913573265076
30 loss: 0.069, accuracy: 0.547
0.6825845837593079
30 loss: 0.068, accuracy: 0.547
0.6925886273384094
30 loss: 0.069, accuracy: 0.547
0.696567177772522
30 loss: 0.070, accuracy: 0.547
0.6879830956459045
30 loss: 0.069, accuracy: 0.547
0.6897417306900024
40 loss: 0.069, accuracy: 0.547
0.6874913573265076
40 loss: 0.069,

In [ ]:
from sklearn.metrics import confusion_matrix
outputs = net0(Variable(torch.from_numpy(X_test.as_matrix()).float()))
_, predicted = torch.max(outputs.data, 1)
confusion = confusion_matrix(y_test, predicted.numpy())
accuracy = accuracy_score(y_test, predicted.numpy())


In [ ]:
confusion

In [ ]:
accuracy